# Coursera Capstone Course - Week 3

### Notebook 1

In [1]:
import pandas as pd
import numpy as np
import folium, json, requests
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

Load the data from csv:

In [2]:
df = pd.read_csv('toronto.csv')

Drop rows where **Borough** cells are *Not assigned*.

In [3]:
df = df.loc[df['Borough']!="Not assigned"]

Fill in any *Not assigned* **Neighborhood** cells with the name of the **Borough**.

In [4]:
df_na = df.loc[df['Neighborhood']=='Not assigned']
df = df.loc[df['Neighborhood']!='Not assigned']
df_na['Neighborhood'] = df_na['Borough']
df = pd.concat([df,df_na],ignore_index=True)

Combine rows with matching **PostalCode** and **Borough** cells, separate **Neighborhood** by a comma.

In [5]:
df['Neighborhood'] = df.groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ', '.join(x))
df.drop_duplicates(inplace=True)

In [6]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
8,M3B,North York,Don Mills North
9,M4B,East York,"Woodbine Gardens, Parkview Hill"
11,M5B,Downtown Toronto,"Ryerson, Garden District"
13,M6B,North York,Glencairn


In [7]:
df.shape

(103, 3)

___

### Notebook 2

In [8]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')

In [9]:
df = df.join(coordinates.set_index('Postal Code'), on='PostalCode')

In [10]:
df.reset_index(drop=True, inplace=True)

In [11]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


---

### Notebook 3

1. Filter the dataframe to only **Boroughs** containing the string *Toronto*.
2. Obtain the coordinates of Toronto for the folium map.
3. Create a folium map with markers on each neighborhood in Toronto.

In [12]:
df = df[df['Borough'].str.contains('Toronto')]

In [13]:
address = 'Toronto'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

4. Access FourSquare to collect and clean the venues in each neighborhood. The function **getNearbyVenues** is lifted from the Segmenting and Clustering Lab.

In [15]:
CLIENT_ID = '152NETJEMBIRNH3M4XHFQR21HK4HLRDK0UCTENTC4LENUNWG'
CLIENT_SECRET = '4T2G3JS23EHDHMQYITOHEMGAXXMT11HOL3ATFZ4PLMZZPCF2'
VERSION = '20180605'
LIMIT = 100

def getNearbyVenues(pc, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for pc, name, lat, lng in zip(pc, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            pc,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode',
                             'Neighborhood', 
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

In [16]:
toronto_venues = getNearbyVenues(pc = df['PostalCode'],
                                 names = df['Neighborhood'],
                                 latitudes = df['Latitude'],
                                 longitudes = df['Longitude'])

Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

5. Clean the data by One-Hot-Encoding the Venue Categories and averaging the grouping of categories by neighborhood.

In [17]:
toronto_venues['Venue Category'] = toronto_venues['Venue Category'].str.replace("Neighborhood","Neighborhoods")  
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
cols = list(toronto_onehot.columns)
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
toronto_onehot['PostalCode'] = toronto_venues['PostalCode']
toronto_onehot = toronto_onehot[['Neighborhood']+['PostalCode']+cols]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,...,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.0,0.01,0.0000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.017857,0.0,0.0,0.0,0.0,0.00,0.0000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0000
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0625
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0000


6. Organize the data by top 10 most common venue category in a neighborhood. The **return_most_common_venues** function is lifted from the Segmenting and Clustering Lab.

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,Clothing Store,Burger Joint,Sushi Restaurant,Asian Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Café,Gourmet Shop
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Breakfast Spot,Grocery Store,Bakery,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Sculpture Garden


---

7. Train a clustering model on the organized data.
8. Visualize the resulting clusters.

In [20]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df.copy()
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(subset=['Cluster Labels'], inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Mexican Restaurant,Café,Restaurant,Event Space,Farmers Market
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Park,Gym,Yoga Studio,Café,Beer Bar,Japanese Restaurant,Juice Bar,Sandwich Place,Burger Joint
8,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Tea Room,Sporting Goods Shop,Bubble Tea Shop,Pizza Place
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Bakery,Diner,Cosmetics Shop,Beer Bar,Cocktail Bar,Hotel,Breakfast Spot
18,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhoods,Pub,Asian Restaurant,Health Food Store,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,The Beaches,0,Neighborhoods,Pub,Asian Restaurant,Health Food Store,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Harbourfront,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Mexican Restaurant,Café,Restaurant,Event Space,Farmers Market
4,Queen's Park,1,Coffee Shop,Park,Gym,Yoga Studio,Café,Beer Bar,Japanese Restaurant,Juice Bar,Sandwich Place,Burger Joint
8,"Ryerson, Garden District",1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Tea Room,Sporting Goods Shop,Bubble Tea Shop,Pizza Place
14,St. James Town,1,Coffee Shop,Café,Restaurant,Bakery,Diner,Cosmetics Shop,Beer Bar,Cocktail Bar,Hotel,Breakfast Spot
19,Berczy Park,1,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Café,Gourmet Shop
23,Central Bay Street,1,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Ice Cream Shop,Sandwich Place,Juice Bar,Burger Joint,Bar,Department Store
24,Christie,1,Grocery Store,Café,Park,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Diner,Candy Store,Coffee Shop
29,"Adelaide, King, Richmond",1,Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,Clothing Store,Burger Joint,Sushi Restaurant,Asian Restaurant,Hotel
30,"Dovercourt Village, Dufferin",1,Bakery,Pharmacy,Grocery Store,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Pizza Place,Café,Brewery,Bar
35,"Harbourfront East, Toronto Islands, Union Station",1,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Scenic Lookout,Sporting Goods Shop,Fried Chicken Joint,Brewery,Plaza


In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Roselawn,2,Garden,Yoga Studio,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,"Forest Hill North, Forest Hill West",3,Park,Jewelry Store,Trail,Sushi Restaurant,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
90,Rosedale,3,Park,Playground,Trail,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,"Moore Park, Summerhill East",4,Playground,Yoga Studio,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### 5 Clusters

- Cluster 0: Size 1 - Mostly residential with businesses which support daily life.
- Cluster 1: Size 34 - Coffee!!! Is this cluster identifying Tim Horton's and Starbuck's locations?!
- Cluster 2: Size 1 - Hip part of town.
- Cluster 3: Size 2 - Green space!
- Cluster 4: Size 1 - Playgroud.

### Conclusion:

The data needs a lot more processing before being thrown into a clustering classifier.